<a href="https://colab.research.google.com/github/HarryKao1020/stock_trading_strategy/blob/colab/backtesting/%E9%A3%86%E8%82%A1%E5%9B%9E%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install backtesting
!pip install shioaji
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from matplotlib.dates import date2num
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from google.colab import userdata

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

#==== for Colab
shioaji_secret=userdata.get('SHIOAJI_SECRETKEY')
shioaji_apikey=userdata.get('SHIOAJI_APIKEY')

# 登入你的key
accounts = api.login(shioaji_apikey, shioaji_secret)



Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [15]:
# 日k的資料

def get_daily_kbar(stock_code, start_date, end_date):
  kbars = api.kbars(
    contract=api.Contracts.Stocks[stock_code],
    start=start_date,
    end=end_date,
  )
  df = pd.DataFrame({**kbars})
  df.ts = pd.to_datetime(df.ts)

  # 依照日期分組，並計算每組的Open, Close, High, Low
  daily_kbar = df.resample('D', on='ts').agg({
      'Open': 'first',
      'Close': 'last',
      'High': 'max',
      'Low': 'min',
      'Volume': 'sum'  # 計算每日成交量的總和
  }).dropna()  # 移除空白的天數
  # 計算當日漲跌幅
  # 取得前一天的Close價
  daily_kbar['Previous_Close'] = daily_kbar['Close'].shift(1)

  # 計算今天的漲跌幅 (今天的Close - 昨天的Close) / 昨天的Close * 100
  daily_kbar['Change_Percentage'] = ((daily_kbar['Close'] - daily_kbar['Previous_Close']) / daily_kbar['Previous_Close'] * 100).round(2)

  # 計算當日震幅
  daily_kbar['Amplitude'] = ((daily_kbar['High'] - daily_kbar['Low']) / daily_kbar['High'] * 100).round(2)

  # 移除前一天沒有數據的行
  daily_kbar = daily_kbar.dropna(subset=['Previous_Close'])
  # 將結果輸出到Excel檔案
  daily_kbar.to_excel(f'{stock_code}-daily_kbar.xlsx', index=True)  # `index=True` 會將日期作為列索引輸出
  return daily_kbar


In [16]:
start_day ="2020-01-01"
end_day = "2020-12-31"

goodStockList2020=[4743,6443,6531,4128,3545 ,6547,2609 ,8046 ,3218 ,6492 ,2368 ,2603 ,4961 ,2303 ,2615 ,3661 ,5269 ,4142 ,6415, 3374]
for(stock_code)in goodStockList2020:
    stock_code = str(stock_code)
    get_daily_kbar(stock_code,start_day,end_day)

# get_daily_kbar(f"{stock_code}",start_day,end_day)

4743
6443
6531
4128
3545
6547
2609
8046
3218
6492
2368
2603
4961
2303
2615
3661
5269
4142
6415
3374


In [17]:
print(api.usage())

connections=1 bytes=286908392 limit_bytes=524288000 remaining_bytes=237379608
